In [ ]:
# Cell 1: Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import (precision_score, recall_score, f1_score, roc_auc_score,
                             average_precision_score, precision_recall_curve, roc_curve,
                             confusion_matrix, classification_report)

In [ ]:
# Cell 2: Load models and test data from 02_modeling notebook
# Run this after executing 02_modeling.ipynb

# These variables come from 02_modeling.ipynb - run that notebook first
# log_reg, dt, rf, svm, gb, X_test_clean, y_test

print("Make sure you have run 02_modeling.ipynb first!")
print(f"Test set size: {X_test_clean.shape}")
print(f"Test label distribution:\n{y_test.value_counts()}")

In [ ]:
# Cell 3: Evaluation Function

def evaluate(model, X_test, y_test, model_name="Model"):
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    print(f"\n{'='*60}")
    print(f"{model_name} Evaluation")
    print(f"{'='*60}")
    print(f"Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"Recall:    {recall_score(y_test, y_pred):.4f}")
    print(f"F1 Score:  {f1_score(y_test, y_pred):.4f}")
    print(f"ROC-AUC:   {roc_auc_score(y_test, y_prob):.4f}")
    print(f"PR-AUC:    {average_precision_score(y_test, y_prob):.4f}")
    print(f"\nConfusion Matrix:\n{confusion_matrix(y_test, y_pred)}")
    print(f"\n{classification_report(y_test, y_pred)}")

    # Plot curves side by side
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    axes[0].plot(fpr, tpr)
    axes[0].set_title(f"ROC Curve — {model_name}")
    axes[0].set_xlabel("FPR"); axes[0].set_ylabel("TPR")

    prec, rec, _ = precision_recall_curve(y_test, y_prob)
    axes[1].plot(rec, prec)
    axes[1].set_title(f"PR Curve — {model_name}")
    axes[1].set_xlabel("Recall"); axes[1].set_ylabel("Precision")

    plt.tight_layout()
    plt.show()

In [ ]:
evaluate(log_reg, X_test_clean, y_test, "Logistic Regression")

In [ ]:
evaluate(dt, X_test_clean, y_test, "Decision Tree")

In [ ]:
evaluate(rf, X_test_clean, y_test, "Random Forest")

## 4. SVM Evaluation

In [ ]:
evaluate(svm, X_test_clean, y_test, "SVM")

## 5. Gradient Boosting Evaluation

In [ ]:
evaluate(gb, X_test_clean, y_test, "Gradient Boosting")

## Model Comparison Report

In [ ]:
# Summary Report for All 5 Models

def generate_report(models_dict, X_test, y_test):
    """Generate a comparison report for multiple models."""
    results = []
    
    for name, model in models_dict.items():
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1]
        
        results.append({
            "Model": name,
            "Precision": precision_score(y_test, y_pred),
            "Recall": recall_score(y_test, y_pred),
            "F1 Score": f1_score(y_test, y_pred),
            "ROC-AUC": roc_auc_score(y_test, y_prob),
            "PR-AUC": average_precision_score(y_test, y_prob)
        })
    
    return pd.DataFrame(results).round(4)

# All 5 models
report_models = {
    "Logistic Regression": log_reg,
    "Decision Tree": dt,
    "Random Forest": rf,
    "SVM": svm,
    "Gradient Boosting": gb
}

print("=" * 60)
print("MODEL COMPARISON REPORT (All 5 Models)")
print("=" * 60)
report_df = generate_report(report_models, X_test_clean, y_test)
display(report_df)

In [ ]:
# ROC and PR Curves Comparison

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
for name, model in report_models.items():
    y_prob = model.predict_proba(X_test_clean)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    auc = roc_auc_score(y_test, y_prob)
    plt.plot(fpr, tpr, label=f"{name} (AUC={auc:.3f})")
plt.plot([0, 1], [0, 1], 'k--', alpha=0.5)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves Comparison")
plt.legend(loc="lower right", fontsize=8)

plt.subplot(1, 2, 2)
for name, model in report_models.items():
    y_prob = model.predict_proba(X_test_clean)[:, 1]
    prec, rec, _ = precision_recall_curve(y_test, y_prob)
    pr_auc = average_precision_score(y_test, y_prob)
    plt.plot(rec, prec, label=f"{name} (PR-AUC={pr_auc:.3f})")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curves Comparison")
plt.legend(loc="lower left", fontsize=8)

plt.tight_layout()
plt.show()